<a href="https://colab.research.google.com/github/amara929/amara929/blob/main/Chat_bot_using_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Step1**:Set Up Environment

In [ ]:
!pip install tensorflow numpy pandas

# Step 2: **Import** libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Step 3: Prepare Dataset

In [ ]:
data = {
    'questions': [
        'hi',
        'how are you',
        'what is your name',
        'i need help',
        'thank you',
        'bye',
        'what is fish farming',
        'how do i start fish farming',
        'which fish is best for farming',
        'how much space do i need for fish farming',
        'what do farmed fish eat',
        'how often should i feed my fish',
        'what is the ideal water temperature for fish',
        'do fish need oxygen in the water',
        'how do i maintain water quality',
        'can i farm fish in tanks',
        'what is the best time to harvest fish',
        'how do i prevent fish diseases',
        'is fish farming profitable',
        'what equipment do i need for fish farming'
    ],
    'answers': [
        'hello!',
        'i am fine, how can i help you?',
        'i am your fish farming assistant.',
        'sure, tell me your issue.',
        'you’re welcome!',
        'goodbye!',
        'fish farming is the process of raising fish in tanks or enclosures for commercial purposes.',
        'to start fish farming, you need proper space, water supply, and fish species suited to your area.',
        'tilapia, catfish, and carp are commonly farmed due to their fast growth and hardiness.',
        'it depends on the fish type, but a small tank setup can work for beginners.',
        'farmed fish eat commercial pellets, live feed, or natural aquatic organisms.',
        'generally, feed them once or twice a day, depending on their size and age.',
        'most fish prefer temperatures between 24°C and 30°C.',
        'yes, oxygen is crucial for fish survival; aeration may be needed.',
        'test water regularly for pH, ammonia, and nitrites, and change water when needed.',
        'yes, tank-based systems are common and easy to control.',
        'harvest time depends on the species but usually occurs when fish reach market size.',
        'keep tanks clean, avoid overfeeding, and monitor water quality to prevent diseases.',
        'yes, with good management, fish farming can be a profitable business.',
        'you will need tanks or ponds, water pumps, aerators, nets, and feeding systems.'
    ]
}

df = pd.DataFrame(data)

# Step 4: Tokenization and Padding

In [ ]:
# Step 4: Tokenize the Questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['questions'])
questions_seq = tokenizer.texts_to_sequences(df['questions'])


# Pad sequences
max_len = max(len(seq) for seq in questions_seq)
questions_seq = pad_sequences(questions_seq, maxlen=max_len, padding='post')

# Prepare labels (as class indexes)
labels = np.arange(len(df))  # [0, 1, 2, ..., 19]


# Step 5: Build the GRU Model

In [ ]:
# Step 5: Build the Model
from tensorflow.keras.layers import GlobalAveragePooling1D # Import GlobalAveragePooling1D
vocab_size = len(tokenizer.word_index) + 1

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(len(df), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d             │ ?                           │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# Step 7: Train The Model

In [ ]:
# Step 6: Train the Model
model.fit(questions_seq, labels, epochs=200, verbose=0)


# Step 8:Create a Response Function

In [ ]:
# Step 7: Build the Response Function
def get_response(user_input):
    seq = tokenizer.texts_to_sequences([user_input])
    seq = pad_sequences(seq, maxlen=max_len, padding='post')
    pred = model.predict(seq, verbose=0)
    idx = np.argmax(pred)
    return df['answers'][idx]

In [ ]:
# Step 8: Run Chat
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        break
    response = get_response(user_input)
    print("Bot:", response)

You: hi
Bot: hello!
